## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-26-16-53-48 +0000,bbc,Trump vs the Fed: Why this row could rattle th...,https://www.bbc.com/news/articles/clydvlx504eo...
1,2025-08-26-16-53-34 +0000,nypost,Gutsy mermaid to tackle treacherous shark-inha...,https://nypost.com/2025/08/26/us-news/fleur-so...
2,2025-08-26-16-51-46 +0000,nyt,"Fed Governor Lisa Cook to Sue Over Firing, Ass...",https://www.nytimes.com/2025/08/26/us/politics...
3,2025-08-26-16-51-44 +0000,nyt,The Deadly Risks of Reporting In Gaza,https://www.nytimes.com/2025/08/26/world/middl...
4,2025-08-26-16-50-13 +0000,startribune,"Live: From ballet to bagpipes, a full day of S...",https://www.startribune.com/live-music-enterta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2376/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,69
65,new,16
25,cook,15
34,gaza,13
154,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...,165
126,2025-08-26-09-30-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,148
26,2025-08-26-16-22-37 +0000,nyt,Can Trump Fire a Fed Governor? What to Know as...,https://www.nytimes.com/2025/08/22/us/politics...,145
32,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,139
25,2025-08-26-16-22-48 +0000,nyt,Lisa Cook Defies Trump Bid to Fire Her as Fed ...,https://www.nytimes.com/2025/08/26/us/politics...,139


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,165,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...
90,58,2025-08-26-11-52-00 +0000,wsj,Illinois Democratic Gov. JB Pritzker promised ...,https://www.wsj.com/politics/policy/chicago-vo...
74,56,2025-08-26-13-35-35 +0000,nypost,Netanyahu calls deadly double strike on Gaza h...,https://nypost.com/2025/08/26/world-news/netan...
104,51,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
298,48,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...
277,39,2025-08-25-19-11-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
97,39,2025-08-26-11-01-03 +0000,nypost,Tragic new photos reveal Idaho murders victim’...,https://nypost.com/2025/08/26/us-news/xana-ker...
268,37,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...
12,34,2025-08-26-16-37-38 +0000,nypost,Trump threatens more tariffs on nations that t...,https://nypost.com/2025/08/26/business/trump-t...
62,34,2025-08-26-14-49-02 +0000,nypost,Attorneys general warn AI firms that ‘if you k...,https://nypost.com/2025/08/26/business/44-us-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
